### Classes

The `class` keyword introduced in ES2015 is just a syntactical sugar. JavaScript has a prototypal inheritance model more powerful than the classical OOP model, which is just a particular use case of it.

In [1]:
class ZooError extends Error {

    // we are poor people, we don't have enum in JavaScript
    static ERROR_CODE = {
        ABSTRACT_INSTANCE_NOT_ALLOWED: 'ABSTRACT_INSTANCE_NOT_ALLOWED',
        METHOD_NOT_IMPLEMENTED: 'METHOD_NOT_IMPLEMENTED',
        ZOMBIE_NOT_ALLOWED: 'ZOMBIE_NOT_ALLOWED'
    };

    // this object has computed keys, instead of hardcoded strings
    static ERROR_MESSAGE = {
        [ZooError.ERROR_CODE.ABSTRACT_INSTANCE_NOT_ALLOWED]: 'You must extend this class',
        [ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED]: 'You must implement this method',
        [ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED]: 'This animal is too old to be alive'
    };

    constructor({ code, message = ZooError.ERROR_MESSAGE[code], data }) {
        super(message);

        this.code = code;
        this.data = data;
    }

    toString() {

        return JSON.stringify({
            code: this.code,
            message: this.message,
            data: this.data
        });
    }
}

Let's define an abstract base class (spoiler: there is no such thing in JavaScript).

In [ ]:
// Restart Kernel & Clear Output after running me
class Animal {

    static maxAge = 100;

    // there is no such thing as protected constructor in JavaScript
    constructor({ birthDate = new Date(), name = 'unknown', species = 'unknown' } = {}) {

        // shower cabins are transparent in JavaScript, the _ is just a sticker that says "don't look"
        this._birthDate = birthDate;
        this._name = name;
        this._species = species;

        if (this.age > Animal.maxAge) {

            throw new ZooError({
                code: ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED,
                data: {
                    species: this._species,
                    age: this.age,
                    maxAge: Animal.maxAge
                }
            });
        }
    }

    // this is a getter (for a made up property)
    get age() {

        const ageInMilliseconds = new Date() - this._birthDate
        return Math.floor(ageInMilliseconds / 1000 / 60 / 60 / 24 / 365); // convert to years
    }

    // we have to manually throw error for unimplemented method
    run() {

        throw new ZooError({
            code: ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED,
            // methods are functions and all functions have the 'name' property
            data: { method: this.run.name }
        });
    }
}

In a sane language you are not allowed to instantiate an abstract base class, nor to change the private properties of an instance.

In [ ]:
{
    const animal = new Animal({ birthDate: new Date('10-10-1980') });
    animal._birthDate = new Date('10-10-1900');

    console.log(`age = ${animal.age}`);

    try {
        animal.run();
    } catch (err) {
        console.log(err);
    }
}

In [ ]:
try {
    const deadAnimal = new Animal({ birthDate: new Date('10-10-1880') });
} catch (err) {
    console.log(err);
}

We can play with the public fields (that should be private) to break the implementation for all other users of this class.

In [ ]:
Animal.maxAge = 0;

try {
    const deadAnimal = new Animal({ birthDate: new Date('10-10-2020') });
} catch (err) {
    console.log(err);
}

Let's make use of modern language features to implement a better abstract base class.

In [2]:
class Animal {

    // private properties, introduced in ES2022 (fresh shit)
    #birthDate;
    #name;
    #species;

    static #maxAge = 100;

    constructor({ birthDate = Date.now(), name = 'unknown', species = 'unknown' }) {

        // this is a pseudo-property that allows to detect if constructor was called using the 'new' operator
        if (new.target === Animal) {

            throw new ZooError({ 
                code: ZooError.ERROR_CODE.ABSTRACT_INSTANCE_NOT_ALLOWED,
                // classes are actually functions behind the scenes and all functions have the 'name' property
                data: { class: Animal.name }
            });
        }

        this.#birthDate = birthDate;
        this.#name = name;
        this.#species = species;

        if (this.age > Animal.#maxAge) {

            throw new ZooError({
                code: ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED,
                data: {
                    species: this.#species,
                    age: this.age,
                    maxAge: Animal.#maxAge
                }
            });
        }
    }

    get birthDate() {
        return this.#birthDate;
    }

    get name() {
        return this.#name;
    }

    set name(name) {
        this.#name = name;
    }

    get species() {
        return this.#species;
    }

    get age() {

        const ageInMilliseconds = new Date() - this._birthDate
        return Math.floor(ageInMilliseconds / 1000 / 60 / 60 / 24 / 365); // convert to years
    }

    run() {

        throw new ZooError({
            code: ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED,
            data: { method: this.run.name }
        });
    }

    greet() {
        console.log(`Hi, I am ${this.#name}`);
    }
}

We also need to extend the base class to implement the required method.

In [3]:
class Dog extends Animal {
    
    constructor({ birthDate, name }) {
        super({ birthDate, name, species: 'dog' });
    }
    
    // implement required method
    run() {
        console.log('Du-te Dica!');
    }

    // override method
    greet() {
        console.log('Bark!');
        
        super.greet();
    }
}

Now we can no longer create an abstract instance directly.

In [4]:
try {
    const animal = new Animal({ birthDate: new Date('01-01-20015'), name: 'Rex' });
} catch (err) {
    console.log(err);
}

ZooError: You must extend this class
    at new Animal (evalmachine.<anonymous>:15:19)
    at evalmachine.<anonymous>:2:20
    at Script.runInThisContext (node:vm:131:12)
    at Object.runInThisContext (node:vm:308:38)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (node:events:365:28)
    at emit (node:internal/child_process:920:12)
    at processTicksAndRejections (node:internal/process/task_queues:84:21) {
  code: 'ABSTRACT_INSTANCE_NOT_ALLOWED',
  data: { class: 'Animal' }
}


In [5]:
{
    const dog = new Dog({ birthDate: new Date('01-01-20015'), name: 'Rex' });
    dog.greet();
    dog.run();
}

Bark!
Hi, I am Rex
Du-te Dica!


Since we now have private properties, we can no longer break the implementation.

In [6]:
try {
    Animal.#maxAge = 0;
} catch (err) {
    console.log(err);
}

SyntaxError: Private field '#maxAge' must be declared in an enclosing class